## Imports

In [ ]:
!pip install geopandas
!pip install shapely
!pip install altair
!pip install pandas
!pip install requests
!pip install vl-convert-python
!pip install python-dotenv
!pip install geopy requests

In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import altair as alt
from shapely.geometry import shape
import datetime
import os
import json
from shapely.geometry import shape, Point
import vl_convert 
from dotenv import load_dotenv
from geopy.geocoders import Nominatim
from shapely.geometry import shape
import isochrone_api as isapi

In [ ]:
#if .env doesn't exist, create it with a placeholder token
if not os.path.exists(".env"):
    with open(".env", "w") as f:
        f.write("NAVITIA_TOKEN=your_navitia_token_here")
    print(".env file created with placeholder tokens. Please update it with your actual API tokens.")
else:
    print(".env file already exists.")

## Data Processing

In [ ]:
INPUT_GEODATA = "../geodata/logement-encadrement-des-loyers.geojson"
ISOCHRONE_PATH = "../isochrone/"
VISUALS_SAVED = "../visuals/"

In [ ]:

paris_zones = gpd.read_file(INPUT_GEODATA) #loads GeoData (GeoJSON) into Geodataframe


# Projection shenaningans
# Calculate the center of each zone (useful for checking if the zone is inside the isochrone)
# We use a projected CRS (like EPSG:3857) to accurately calculate centroids, then convert back to GPS (EPSG:4326)
paris_zones['centroid'] = paris_zones.to_crs(epsg=3857).centroid.to_crs(epsg=4326)

paris_zones['commute_minutes'] = 60 #column for commute time, initialized at 60 (minutes)

In [ ]:
#transform 'annee' colum, into numeric values (integers), to be able to filter by year
paris_zones['annee'] = pd.to_numeric(paris_zones['annee'], errors='coerce') 

#keep only data of years 2024 and onward
paris_zones = paris_zones[paris_zones['annee'] >= 2024]



In [ ]:
display(paris_zones.head())

## Test Map
This part is for testing the feasibility of using altair for map data display.

In [ ]:
# --- Altair Data Prep ---
# We drop 'centroid' so Altair's JSON serializer doesn't crash on the Point objects
display_data = paris_zones.drop(columns=['centroid'])

geojson_data = alt.InlineData(
    values=display_data.__geo_interface__, 
    format=alt.DataFormat(property='features', type='json')
)

# --- Interactive Map ---
interactive_map = alt.Chart(geojson_data).mark_geoshape(
    stroke='white',
    strokeWidth=0.5
).encode(
    color=alt.Color(
        'properties.nom_quartier:N', 
        legend=None, 
        scale=alt.Scale(scheme='category20b') 
    ),
    tooltip=[
        alt.Tooltip('properties.nom_quartier:N', title='Neighborhood'),
        alt.Tooltip('properties.id_quartier:N', title='Zone Number'),
        alt.Tooltip('properties.ref:Q', title='Reference Rent (€/m²)'),
        alt.Tooltip('properties.commute_minutes:N', title='Commute Time')
    ]
).properties(
    width=800,
    height=600,
    title="Map of Paris Neighborhoods"
).project(
    type='mercator' 
)
#saved as png file
interactive_map.save(os.path.join(VISUALS_SAVED, "paris_neighborhoods_map.png"))
interactive_map

### By rent price

We can have mltiple types of visualisation:

#### 1. Rent intervals
A student would try to search potential districts on the following criteras:
- minimum rent price 
- maximum rent price

Then a map should show:
- districts that fall into this interval. Other districts would be greyed out.

### 2. Rent rankings
A simpler visual representation where a student can gauge at first glance which districts would be better (green to red).

In [ ]:
# intervals map
MIN_RENT = 20
MAX_RENT = 25

In [ ]:
intervals_map = alt.Chart(geojson_data).mark_geoshape(
    stroke='white',
    strokeWidth=0.5
).encode(
    color=alt.condition(
        # Check if the reference rent falls between the user's min and max
        f"datum.properties.ref >= {MIN_RENT} && datum.properties.ref <= {MAX_RENT}",
        alt.value("#3182bd"),    # True: Highlighted in blue
        alt.value("lightgray")   # False: Greyed out
    ),
    tooltip=[
        alt.Tooltip('properties.nom_quartier:N', title='Neighborhood'),
        alt.Tooltip('properties.ref:Q', title='Reference Rent (€/m²)'),
        alt.Tooltip('properties.piece:N', title='Rooms'),
        alt.Tooltip('properties.meuble_txt:N', title='Type')
    ]
).properties(
    width=800,
    height=600,
    title=f"Districts between {MIN_RENT}€ and {MAX_RENT}€ /m²"
).project(
    type='mercator' 
)
intervals_map.save(os.path.join(VISUALS_SAVED, f"paris_rent_intervals_{MIN_RENT}_{MAX_RENT}.png"))
intervals_map


In [ ]:
#just a green to red rent map
rent_map = alt.Chart(geojson_data).mark_geoshape(
    stroke='white',
    strokeWidth=0.5
).encode(
    color=alt.Color(
        'properties.ref:Q', 
        scale=alt.Scale(scheme='redyellowgreen', reverse=True), 
        legend=alt.Legend(title='Reference Rent (€/m²)')
    ),
    tooltip=[
        alt.Tooltip('properties.nom_quartier:N', title='Neighborhood'),
        alt.Tooltip('properties.ref:Q', title='Reference Rent (€/m²)'),
        alt.Tooltip('properties.piece:N', title='Rooms'),
        alt.Tooltip('properties.meuble_txt:N', title='Type')
    ]
).properties(
    width=800,
    height=600,
    title="Reference Rent Map of Paris Neighborhoods"
).project(
    type='mercator' 
)

rent_map.save(os.path.join(VISUALS_SAVED, "paris_rent_map.png"))
rent_map

### Find districts where there are know "furnitured" appartments


In [ ]:
#grey out the districts that do not have "meublé" appartments
furnished_map = alt.Chart(geojson_data).mark_geoshape(
    stroke='white',
    strokeWidth=0.5
).encode(
    color=alt.condition(
        "datum.properties.meuble_txt === 'meublé'",
        alt.value("#2ca02c"),    # True: Highlighted in green
        alt.value("lightgray")   # False: Greyed out
    ),
    tooltip=[
        alt.Tooltip('properties.nom_quartier:N', title='Neighborhood'),
        alt.Tooltip('properties.ref:Q', title='Reference Rent (€/m²)'),
        alt.Tooltip('properties.piece:N', title='Rooms'),
        alt.Tooltip('properties.meuble_txt:N', title='Type')
    ]
).properties(
    width=800,
    height=600,
    title="Districts with existing furnished apartments (green) vs those without (grey) - Paris Neighborhoods"
).project(
    type='mercator' 
)

furnished_map.save(os.path.join(VISUALS_SAVED, "paris_furnished_map.png"))
furnished_map

## Accessibility (Commute time)
This part tackles the question "where should I rent if I am concerned about commute time between my appartment and the campus ?".

The way this part works is by doing the reverse of the question. Using an API, we compute the accessibility of Paris districts FROM a selected zone (the selected campus address).

In [ ]:
CAMPUS_ADDRESS = "47 Rue des Bergers, 75015 Paris, France"
load_dotenv()  # loads variables from .env into environment

NAVITIA_TOKEN = os.getenv("NAVITIA_TOKEN")
print(f"Token loaded: {'✅' if NAVITIA_TOKEN else '❌ Not found'}")
print(f"Navitia Token: {NAVITIA_TOKEN[:4]}...{NAVITIA_TOKEN[-4:]}")  # Print only the first and last 4 characters for verification

In [ ]:
CAMPUS_ADDRESS = "Université Paris 1 Panthéon-Sorbonne, Paris, France"
CAMPUS_NAME = "Paris 1 Panthéon-Sorbonne"
TIME_STUDIED = 30  # minutes

# Skip geocoder — hardcode correct coordinates for Paris 1 (Place du Panthéon)
start_lon = 2.3461
start_lat = 48.8462

print(f"Using hardcoded coordinates for Paris 1: {start_lon}, {start_lat}")

In [ ]:
#if cached file exists, load isochrone from there; otherwise, fetch from Navitia API and save to cache
cached_file = f"isochrone_paris_{TIME_STUDIED}min.json"
if os.path.exists(cached_file):
    print(f"Loading isochrone data from cache: {cached_file}")
    with open(cached_file, "r") as f:
        data = json.load(f)
else:
    print(f"Fetching isochrone data from Navitia API for {TIME_STUDIED}-min commute...")
    isapi.call_API(TIME_STUDIED, NAVITIA_TOKEN, start_lon, start_lat)
    with open(cached_file, "r") as f:
        data = json.load(f)

In [ ]:
#we get all files from the isochrone folder
isochrone_files = [f for f in os.listdir(ISOCHRONE_PATH) if f.endswith(".json")]
display(isochrone_files)

In [ ]:
# --- Load all isochrone files and build a dict of polygons ---
isochrone_polygons = {}  # key: duration in minutes, value: shapely polygon

for iso_file in isochrone_files:
    with open(os.path.join(ISOCHRONE_PATH, iso_file), "r") as f:
        iso_data = json.load(f)
    
    iso = iso_data["isochrones"][0]
    duration_min = iso["max_duration"] // 60  # convert seconds to minutes
    iso_polygon = shape(iso["geojson"])
    isochrone_polygons[duration_min] = iso_polygon
    
    print(f"Loaded: {iso_file} → {duration_min} min isochrone ({iso_polygon.geom_type})")

print(f"\nAvailable durations: {sorted(isochrone_polygons.keys())} minutes")

In [ ]:
# --- Tag each zone with the shortest isochrone it falls into ---
centroids = paris_zones.to_crs(epsg=3857).centroid.to_crs(epsg=4326)

sorted_durations = sorted(isochrone_polygons.keys())  # e.g. [15, 30, 45, 60]

def get_commute_time(centroid):
    """Returns the smallest isochrone duration the centroid falls into, or None if outside all."""
    for duration in sorted_durations:
        if isochrone_polygons[duration].contains(centroid):
            return duration
    return None

paris_zones["commute_minutes"] = centroids.apply(get_commute_time)

# Summary
for d in sorted_durations:
    count = (paris_zones["commute_minutes"] == d).sum()
    print(f"Zones first reached at {d} min: {count}")
unreachable = paris_zones["commute_minutes"].isna().sum()
print(f"Zones outside all isochrones: {unreachable}")

In [ ]:
# --- Map: Paris districts coloured by commute time ---
display_data = paris_zones.drop(columns=["centroid"], errors="ignore")
paris_geojson = alt.InlineData(
    values=display_data.__geo_interface__,
    format=alt.DataFormat(property="features", type="json")
)

paris_base = alt.Chart(paris_geojson).mark_geoshape(
    stroke="white",
    strokeWidth=0.5
).encode(
    color=alt.condition(
        "datum.properties.commute_minutes !== null",
        alt.Color(
            "properties.commute_minutes:O",
            scale=alt.Scale(
                domain=sorted_durations,
                range=["#0984e3", "#74b9ff", "#a29bfe", "#dfe6e9"][:len(sorted_durations)]
            ),
            legend=alt.Legend(title="Commute (min)")
        ),
        alt.value("#e0e0e0")  # grey if outside all isochrones
    ),
    tooltip=[
        alt.Tooltip("properties.nom_quartier:N", title="Neighborhood"),
        alt.Tooltip("properties.commute_minutes:O", title="Commute (min)"),
        alt.Tooltip("properties.ref:Q", title="Reference Rent (€/m²)"),
    ]
)

campus_point = alt.Chart(
    {"values": [{"lon": 2.3463, "lat": 48.8467}]}
).mark_point(color="red", size=100, shape="cross", filled=True).encode(
    longitude="lon:Q",
    latitude="lat:Q",
    tooltip=alt.value("Université Paris 1")
)

durations_str = ", ".join([f"{d} min" for d in sorted_durations])
chart = (paris_base + campus_point).properties(
    width=700,
    height=600,
    title=f"Paris Districts by Commute Time from Université Paris 1 ({durations_str})"
).project(type="mercator")


chart.save(os.path.join(VISUALS_SAVED, "paris_commute_map.png"))
chart

>Results are for Saturday 18:30 (more or less). 